<a href="https://colab.research.google.com/github/michelprosite/olist_brasil/blob/main/olist_brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
os.getcwd()

'/content'

In [7]:
import pandas as pd
import os
import seaborn as sns
import kaggle

OSError: ignored

In [ ]:
def created_folders():
    # Variavel responsável pos capiturar o path raiz do projeto
    path_folder = os.getcwd() 
    # Abrindo e carregando o arquivo CSV com os nomes das pastas que serão criadas para o projeto
    file_path = os.path.join(path_folder, 'nomes_diretorios.csv')
    f = pd.read_csv(file_path)
    lista_folders = f['Folders'].to_list()

    #Criando o diretório "data"
    if not os.path.exists('data'):
            os.makedirs('data')

    # Laço for percorre a lista das partas cridas e inicia as pastas do projeto
    for i in lista_folders:
        diretorio = path_folder

        # Verifica se as pastas já foram criadas, se não, as cria.
        if not os.path.exists(diretorio + '/data/' + i):
            os.makedirs(diretorio + '/data/' + i)
            print(f"Diretório {i} criado com sucesso!")
        else:
            print(f"O diretório {i} já existe.")
created_folders()

In [ ]:
def download_csv_olist():
    folder = os.getcwd()
    path_folder = os.path.join(folder, 'data/transient')

    files = os.listdir(path_folder)

    if len(files) == 0:
        PATH_FOLDER = path_folder
        os.environ['PATH_FOLDER'] = PATH_FOLDER

        kaggle.api.dataset_download_files('olistbr/brazilian-ecommerce', path=PATH_FOLDER, unzip=True)
        print('Arquivos carregados e descompactados.')
    else:
        print()
        print(f'Arquivos já existem e ou foram atualizados!')
        print('Caso queira atualiza-los se faz necessário executar a recarga da Raw para zerar os arquivos.')
download_csv_olist()

In [ ]:
def created_arquivo_controller():
    if os.path.exists(os.getcwd() + '/transient'):
        folder = os.getcwd()
        path_folder = os.path.join(folder, 'data/transient')  # Substitua pelo caminho da pasta desejada
        file_path = os.path.join(folder, 'data/controller.csv')
    else:
        folder = os.getcwd()
        path_folder = os.path.join(folder, 'data/raw')  # Substitua pelo caminho da pasta desejada
        file_path = os.path.join(folder, 'data/controller.csv')

    print(file_path)

    # Capturar nomes dos arquivos e remover a extensão
    arquivos = os.listdir(path_folder)

    # Eliminar as palavras indesejadas
    palavras_indesejadas = ['_dataset.csv', 'olist_', 'order_', '_dataset', 'product_', '_name', '_translation', '.csv', '.parquet']
    nomes_limpos = [arquivo for arquivo in arquivos]
    for palavra in palavras_indesejadas:
        nomes_limpos = [nome.replace(palavra, '') for nome in nomes_limpos]

    # Caminho do arquivo CSV a ser salvo
    caminho_arquivo = str(path_folder[0]) + '/' + 'controller.csv'

    # Definir os dados a serem escritos no arquivo CSV
    dados = [
        ["path_transient", "path_raw", "path_trusted", "table_transient", "table_raw", "table_trusted", "table_name", "table_name_temp"]
    ] + [
        [str(path_folder[0]) + "data/transient/", str(path_folder[0]) + "data/raw/", str(path_folder[0]) + "data/trusted/", nome.replace('.csv', "").replace('.parquet', ""), nome.replace('.csv', "").replace('.parquet', ""), nome.replace('.csv', "").replace('.parquet', ""), nome_limp, 'temp_' + nome_limp]
        for nome, nome_limp in zip(arquivos, nomes_limpos)
    ]

    # Salvar o arquivo CSV
    pd.DataFrame(dados).to_csv(file_path, index=False, header=False)
    print('Arquivo controller criado com sucesso!')
created_arquivo_controller()


In [ ]:
controller = os.path.join(os.getcwd(), 'data/controller.csv')
path = pd.read_csv(controller)
path

In [ ]:
def carregando_raw():
    folder = os.getcwd()
    file_path = os.path.join(folder, 'data/controller.csv')
    path = pd.read_csv(file_path)
    print('Path criado com base no arquivo controller!')

    for i in range(len(path['path_transient'])):
        # Criando o caminho dos arquivos e diretórios
        diretorio_transient = path['path_transient'][i]
        nome_arquivo_transient = path['table_transient'][i] + '.csv'
        caminho_arquivo_transient = (folder + diretorio_transient + nome_arquivo_transient)

        diretorio_raw = path['path_raw'][i]
        nome_arquivo_raw = path['table_raw'][i] + '.parquet'
        caminho_arquivo_raw = (folder + diretorio_raw + nome_arquivo_raw)

        # Verificando a existência dos arquivos em Transient e carrega na Raw pela primeira vez
        if os.path.exists(caminho_arquivo_transient) and not os.path.exists(caminho_arquivo_raw):
            # Carregando arquivos csv
            df = pd.read_csv(caminho_arquivo_transient)

            # Dropando os duplicados preservando o último atualizado
            df.drop_duplicates(subset=df.columns[0], keep='last', inplace=True)

            # Verifica se o DataFrame não está vazio
            if not df.empty:
                # Salva na Raw convertendo para o formato Parquet
                df.to_parquet(caminho_arquivo_raw)
                print(f"Arquivo Parquet {nome_arquivo_raw} criado: {caminho_arquivo_raw}")
            else:
                print(f"O DataFrame está vazio: {nome_arquivo_raw}")

        # Caso a Raw já tenha sido carregada pela primeira vez, esse código é executado para atualizar os arquivos verificando se algum arquivo foi deletado.
        elif os.path.exists(caminho_arquivo_transient) and os.path.exists(caminho_arquivo_raw):
            # Confere a existência do arquivo, caso deletado, cria novamente
            if not os.path.exists(caminho_arquivo_raw):
                df = pd.read_csv(caminho_arquivo_transient)
                df.drop_duplicates(subset=df.columns[0], keep='last', inplace=True)
                if not df.empty:
                    df.to_parquet(caminho_arquivo_raw)
                    print(f"Arquivo Parquet {nome_arquivo_raw} criado: {caminho_arquivo_raw}")

            # Carregando os DataFrames para concatenar os novos dados
            df1 = pd.read_csv(caminho_arquivo_transient)
            df2 = pd.read_parquet(caminho_arquivo_raw)

            # Concatenando os dados
            df_concatenado = pd.concat([df2, df1], axis=0)

            # Eliminando as duplicatas pelo ID mantendo sempre a última ocorrência
            df_concatenado = df_concatenado.drop_duplicates(subset=df_concatenado.columns[0], keep='last')
            df_concatenado.to_parquet(caminho_arquivo_raw)
        else:
            print(f'Arquivo Raw {nome_arquivo_raw} já existe! Para atualizá-lo, carregue a transient correspondente.')

    # Após a criação e atualização dos dados, os arquivos Transient são eliminados deixando o diretório
    # livre para receber novos dados
    print()
    print()
    for i in range(len(path['path_transient'])):
        diretorio_transient = path['path_transient'][i]
        nome_arquivo_transient = path['table_transient'][i] + '.csv'
        caminho_arquivo_transient = (folder + diretorio_transient + nome_arquivo_transient)
        
        if os.path.exists(caminho_arquivo_transient):
            os.remove(caminho_arquivo_transient)

        print(f"Arquivo {file_path} da Transient eliminado.")

carregando_raw()


In [ ]:
def carregando_trusted():
    folder = os.getcwd()
    file_path = os.path.join(folder, 'data/controller.csv')
    path = pd.read_csv(file_path)

    for i in range(len(path['path_raw'])):
        # Criando o caminho dos arquivos e diretórios
        diretorio_raw = path['path_raw'][i]
        nome_arquivo_raw = path['table_raw'][i] + '.parquet'
        caminho_arquivo_raw = (folder + diretorio_raw + nome_arquivo_raw)

        diretorio_trusted = path['path_trusted'][i]
        nome_arquivo_trusted = path['table_trusted'][i] + '.parquet'
        caminho_arquivo_trusted = (folder + diretorio_trusted + nome_arquivo_trusted)

        # Verificando a existência dos arquivos em Transient e carrega na Raw pela primeira vez
        if os.path.exists(caminho_arquivo_raw) and not os.path.exists(caminho_arquivo_trusted):
            # Carregando arquivos parquet
            df = pd.read_parquet(caminho_arquivo_raw)

            # Dropando os duplicados preservando o último atualizado
            df.drop_duplicates(subset=df.columns[0], keep='last', inplace=True)

            # Verifica se o DataFrame não está vazio
            if not df.empty:
                # Salva na Raw convertendo para o formato Parquet
                df.to_parquet(caminho_arquivo_trusted)
                print(f"Arquivo Parquet {nome_arquivo_trusted} criado: {caminho_arquivo_trusted}")
            else:
                print(f"O DataFrame está vazio: {nome_arquivo_trusted}")

        # Caso a Raw já tenha sido carregada pela primeira vez, esse código é executado para atualizar os arquivos verificando se algum arquivo foi deletado.
        elif os.path.exists(caminho_arquivo_raw) and os.path.exists(caminho_arquivo_trusted):
            # Confere a existência do arquivo, caso deletado, cria novamente
            if not os.path.exists(caminho_arquivo_trusted):
                df = pd.read_parquet(caminho_arquivo_raw)
                df.drop_duplicates(subset=df.columns[0], keep='last', inplace=True)
                if not df.empty:
                    df.to_parquet(caminho_arquivo_trusted)
                    print(f"Arquivo Parquet {nome_arquivo_trusted} criado: {caminho_arquivo_trusted}")

            # Carregando os DataFrames para concatenar os novos dados
            df1 = pd.read_parquet(caminho_arquivo_raw)
            df2 = pd.read_parquet(caminho_arquivo_trusted)

            # Concatenando os dados
            df_concatenado = pd.concat([df2, df1], axis=0)

            # Eliminando as duplicatas pelo ID mantendo sempre a última ocorrência
            df_concatenado = df_concatenado.drop_duplicates(subset=df_concatenado.columns[0], keep='last')
            df_concatenado.to_parquet(caminho_arquivo_trusted)
        else:
            print(f'Arquivo Raw {nome_arquivo_trusted} já existe! Para atualizá-lo, carregue a transient correspondente.')
carregando_trusted()


In [ ]:
folder = os.getcwd()
file_path = os.path.join(folder, 'data/controller.csv')
path = pd.read_csv(file_path)
for i in range(len(path['table_name'])):
    print(f"Nome da tabela: {path['table_name'][i]}")
    print(f"caminho dos dados: {path['path_trusted'][i] + path['table_trusted'][i] + '.parquet'}")
    print()

In [ ]:
import mysql.connector

# Configurações de conexão com o banco de dados
host = 'localhost'
user = 'root'
password = '*Santana25'

for i in range(len(path['table_name_temp'])):
    # Conectando ao servidor MySQL
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password
    )

    # Criando um cursor para executar comandos SQL
    cursor = connection.cursor()

    # Verificar se o banco 'olistdb' existe, se não, criá-lo
    cursor.execute("SHOW DATABASES LIKE 'olistdb'")
    result = cursor.fetchone()
    if not result:
        cursor.execute("CREATE DATABASE olistdb")

    # Fechar o cursor
    cursor.close()

    # Conectar ao banco de dados 'olistdb'
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database='olistdb'
    )

    # Capturar os nomes das futuras tabelas em df['table_name_temp']
    df = pd.read_parquet(os.getcwd() + path['path_trusted'][i] + path['table_trusted'][i] + '.parquet')
    table_name_temp = path['table_name_temp'][i]

    # Criar um novo cursor
    cursor = connection.cursor()

    # Verificar se as tabelas existem, se não, criar as tabelas no banco de dados
    cursor.execute(f"SHOW TABLES LIKE '{table_name_temp}'")
    result = cursor.fetchone()
    if not result:
        # Obter as colunas e seus tipos a partir do arquivo Parquet
        table_path = os.getcwd() + path['path_trusted'][i] + path['table_trusted'][i] + '.parquet'
        table_data = pd.read_parquet(table_path)
        columns = table_data.columns
        column_types = table_data.dtypes
        table_index = table_data.index

        # Criar a tabela com as colunas e tipos correspondentes
        create_table_query = f"CREATE TABLE {table_name_temp} ("
        for col, col_type in zip(columns, column_types):
            if col_type == 'int64':
                col_type = 'INT'
            elif col_type == 'float64':
                col_type = 'FLOAT'
            elif col_type == 'bool':
                col_type = 'BOOLEAN'
            else:
                col_type = 'VARCHAR(255)'  # Tipo padrão se não for numérico

            if col == columns[0]:
                create_table_query += f"{col} {col_type} PRIMARY KEY, "
            else:
                create_table_query += f"{col} {col_type}, "

        create_table_query = create_table_query.rstrip(', ')  # Remover a última vírgula
        create_table_query += ")"
        cursor.execute(create_table_query)

    # Fechar o cursor
    cursor.close()

    # Fechar a conexão com o banco de dados
    connection.close()

In [ ]:
# Configurações de conexão com o banco de dados MySQL
config = {
    'user': 'root',
    'password': '*Santana25',
    'host': 'localhost',
    'database': 'olistdb',
    'raise_on_warnings': True
}

# Função para limpar uma tabela
def limpar_tabela(cursor, tabela):
    cursor.execute(f"DELETE FROM {tabela_temp}")

# Função para carregar dados de um arquivo Parquet para uma tabela
def carregar_dados_parquet(cursor, tabela_temp, caminho):
    dataframe = pd.read_parquet(caminho)
    dataframe = dataframe.fillna(0)  # Substituir valores NaN por 0
    colunas = dataframe.columns.tolist()
    placeholders = ','.join(['%s'] * len(colunas))
    
    # Utilizar INSERT INTO ... VALUES (...) AS alias e substituir VALUES(col) pelo alias.col na cláusula ON DUPLICATE KEY UPDATE
    insert_query = f"INSERT INTO {tabela_temp} ({', '.join(colunas)}) VALUES ({placeholders}) AS tmp " \
                  f"ON DUPLICATE KEY UPDATE " \
                  f"{', '.join([f'{column}=tmp.{column}' for column in colunas])}"
    
    valores = [tuple(row) for row in dataframe.values]
    cursor.executemany(insert_query, valores)

# Conectando ao banco de dados
try:
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()

    # Verificando e carregando tabelas
    for i in range(len(path['table_name_temp'])):
        tabela_temp = path['table_name_temp'][i]
        caminho = os.getcwdb().decode() + path['path_trusted'][i] + path['table_trusted'][i] + '.parquet'
        
        # Limpar tabela antes de carregar os dados
        limpar_tabela(cursor, tabela_temp)
        
        carregar_dados_parquet(cursor, tabela_temp, caminho)
        cnx.commit()
        print(f"Dados carregados na {tabela_temp} com sucesso!")

except mysql.connector.Error as err:
    print(f"Erro ao conectar ao banco de dados: {err}")

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'cnx' in locals() and cnx.is_connected():
        cnx.close()

In [ ]:
import mysql.connector

# Configurações de conexão com o banco de dados
host = 'localhost'
user = 'root'
password = '*Santana25'

for i in range(len(path['table_name'])):
    # Conectando ao servidor MySQL
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password
    )

    # Criando um cursor para executar comandos SQL
    cursor = connection.cursor()

    # Verificar se o banco 'olistdb' existe, se não, criá-lo
    cursor.execute("SHOW DATABASES LIKE 'olistdb'")
    result = cursor.fetchone()
    if not result:
        cursor.execute("CREATE DATABASE olistdb")

    # Fechar o cursor
    cursor.close()

    # Conectar ao banco de dados 'olistdb'
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database='olistdb'
    )

    # Capturar os nomes das futuras tabelas em df['table_name']
    df = pd.read_parquet(os.getcwd() + path['path_trusted'][i] + path['table_trusted'][i] + '.parquet')
    table_name = path['table_name'][i]

    # Criar um novo cursor
    cursor = connection.cursor()

    # Verificar se as tabelas existem, se não, criar as tabelas no banco de dados
    cursor.execute(f"SHOW TABLES LIKE '{table_name}'")
    result = cursor.fetchone()
    if not result:
        # Obter as colunas e seus tipos a partir do arquivo Parquet
        table_path = os.getcwdb().decode() + path['path_trusted'][i] + path['table_trusted'][i] + '.parquet'
        table_data = pd.read_parquet(table_path)
        columns = table_data.columns
        column_types = table_data.dtypes
        table_index = table_data.index

        # Criar a tabela com as colunas e tipos correspondentes
        create_table_query = f"CREATE TABLE {table_name} ("
        for col, col_type in zip(columns, column_types):
            if col_type == 'int64':
                col_type = 'INT'
            elif col_type == 'float64':
                col_type = 'FLOAT'
            elif col_type == 'bool':
                col_type = 'BOOLEAN'
            else:
                col_type = 'VARCHAR(255)'  # Tipo padrão se não for numérico

            if col == columns[0]:
                create_table_query += f"{col} {col_type} PRIMARY KEY, "
            else:
                create_table_query += f"{col} {col_type}, "

        create_table_query = create_table_query.rstrip(', ')  # Remover a última vírgula
        create_table_query += ")"
        cursor.execute(create_table_query)

    # Fechar o cursor
    cursor.close()

    # Fechar a conexão com o banco de dados
    connection.close()

In [ ]:
# Configurações de conexão com o banco de dados MySQL
config = {
    'user': 'root',
    'password': '*Santana25',
    'host': 'localhost',
    'database': 'olistdb',
    'raise_on_warnings': True
}

# Função para verificar se uma tabela está vazia
def tabela_vazia(cursor, tabela):
    cursor.execute(f"SELECT COUNT(*) FROM {tabela}")
    result = cursor.fetchone()
    return result[0] == 0

# Função para carregar dados de um arquivo Parquet para uma tabela
def carregar_dados_parquet(cursor, tabela, caminho):
    dataframe = pd.read_parquet(caminho)
    dataframe = dataframe.fillna(0)  # Substituir valores NaN por 0
    valores = [tuple(row) for row in dataframe.values]
    placeholders = ','.join(['%s'] * len(dataframe.columns))
    cursor.executemany(f"INSERT INTO {tabela} VALUES ({placeholders})", valores)

# Conectando ao banco de dados
try:
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()

    # Verificando e carregando tabelas
    for i in range(len(path['table_name'])):
        tabela = path['table_name'][i]
        if tabela_vazia(cursor, tabela):
            caminho = os.getcwdb().decode() + path['path_trusted'][i] + path['table_trusted'][i] + '.parquet'
            carregar_dados_parquet(cursor, tabela, caminho)
            cnx.commit()
            print(f"Dados carregados na tabela {tabela} com sucesso!")
        else:
            print(f"A tabela {tabela} já possui dados e não será carregada.")

except mysql.connector.Error as err:
    print(f"Erro ao conectar ao banco de dados: {err}")

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'cnx' in locals() and cnx.is_connected():
        cnx.close()

In [ ]:
# Configurações de conexão com o banco de dados MySQL
config = {
    'user': 'root',
    'password': '*Santana25',
    'host': 'localhost',
    'database': 'olistdb',
    'raise_on_warnings': True
}

# Código SQL
sql_update = """
UPDATE `category` AS destino
JOIN `temp_category` AS origem ON destino.`product_category_name` = origem.`product_category_name`
SET destino.`product_category_name` = origem.`product_category_name`,
    destino.`product_category_name` = origem.`product_category_name`
"""

sql_insert = """
INSERT INTO `category` (`product_category_name`)
SELECT origem.`product_category_name`
FROM `temp_category` AS origem
LEFT JOIN `category` AS destino ON origem.`product_category_name` = destino.`product_category_name`
WHERE destino.`product_category_name` IS NULL
"""

sql_delete = """
DELETE destino
FROM `category` AS destino
LEFT JOIN `temp_category` AS origem ON destino.`product_category_name` = origem.`product_category_name`
WHERE origem.`product_category_name` IS NULL
"""

try:
    # Conectando ao banco de dados
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()

    # Executando as consultas SQL
    cursor.execute(sql_update)
    print("UPDATE executado com sucesso!")

    cursor.execute(sql_insert)
    print("INSERT executado com sucesso!")

    cursor.execute(sql_delete)
    print("DELETE executado com sucesso!")

    cnx.commit()

except mysql.connector.Error as err:
    print(f"Erro ao conectar ao banco de dados: {err}")

finally:
    if cursor:
        cursor.close()
    if cnx.is_connected():
        cnx.close()

****************************


[Ontem 15:05] Rodrigo Aparecido Kartcheski




primeiro passo

* achar as primary key

 

segundo passo

* achar quais tabelas se relacionam - buscar relacionametno entre fatos e dimensões - fazer o desenho da modelagem

 

terceiro passo

* fazer join das tabelas

 

quarto passo

* fazer a carga dos dados usando (CRUD)


